# Part 2: The objective of this part is to join in the coordinates to the previous dataframe

#### Importing the code from the previous part of the assignment

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

#Importing libraries
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')

#Webscraping to Dataframe
toronto_lst = []
for row in soup.find('table').find_all('tr'):
    line = row.find_all('td')
    if len(line)!=0:
        toronto_lst.append([line[0].text.replace("\n", ""), line[1].text.replace("\n", ""), line[2].text.replace("\n", "")])
df = pd.DataFrame.from_records(toronto_lst).rename(columns={0:'PostalCode',1:'Borough',2:'Neighborhood'})

#Removing Boroughs with not assigned
df = df[df['Borough']!='Not assigned'].reset_index(drop=True)

#Merging the boroughs with multiple postal code
df = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))

#Assign neighborhood name as the borough is neighborhood is not assigned
for _,row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

#Display the shape of the dataframe
df.shape

(103, 3)

#### Generating latitude and Longitude from csv file

Note: The csv import was done using IBM Watson which publishes the key. For the sake of security, the cell with the key was hidden.

In [28]:
df_coordinates = pd.read_csv(body)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging the two dataframes together to get the same table with all the information

Note: The coordinates dataframe was changed column names to ensure that there are no duplicate columns and that the merge is done cleanly.

In [35]:
df_coordinates = df_coordinates.rename(columns={'Postal Code':'PostalCode'})
df_toronto = df.merge(df_coordinates,left_on='PostalCode',right_on='PostalCode')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Checking if same as assignment

In [34]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
df_test = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    df_test = df_test.append(df_toronto[df_toronto["PostalCode"]==postcode], ignore_index=True)
    
df_test

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
